# 利用1_1的stan模型生成数据

In [6]:
import pystan
%load_ext jupyterstan

import numpy as np
import arviz as az
import xarray as xr
from math import pi

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from stan_utils import *

The jupyterstan extension is already loaded. To reload it, use:
  %reload_ext jupyterstan


#  要用到的函数

In [10]:
def gen_transmmition_data(i_group_num, i_random_seed,l_trans_power):
    '''
    该函数随机生成生成发送节点的x，y坐标值和发送功率
    Parameters
        ----------
        group_num: int,数据的组数
        random_seed: int,随机数种子
        l_trans_power: iterable,功率值
    Returns
        ----------
        transmmition_data: xarray格式,dim = (group, var:[x,y,power])
    '''
    if (len(l_trans_power) != i_group_num):
        raise ValueError("功率值列表长度应与数据组数一致")
    data = np.zeros((i_group_num, 3))

    np.random.seed(i_random_seed)
    data[:,0] = 0.5 + np.random.rand(i_group_num)*(499.5-0.5)
    data[:,1] = -499.5 + np.random.rand(i_group_num)*(-0.5+499.5)
    trans_power = np.array([i for i in l_trans_power])
    data[:,2] = trans_power
    transmmition_data = xr.DataArray(data,
                              dims=('group', 'var'),
                              coords={'var':['x','y','power']}
                             )
    return transmmition_data

def recv_power(xr_transmmiton_data, coord_x, coord_y, r_η, fc=700e6):
    '''
    该函数根据transmmiton_data，节点坐标recv_x, recv_y, 自由空间传播损耗因子η，生成每个节点的接收功率
    Parameters
        ----------
        xr_transmmion_data: xarray,发送节点数据,dims{var,group}
        recv_x: np.meshgrid生成的x坐标
        recv_y: np.meshgrid生成的y坐标
        r_η: real,自由空间传播损耗因子
    Returns
        ----------
        spectrum_data: xarray格式<xarray.DataArray (y: 500, x: 500, group: n)>    
    '''
    lamda = 3e8/fc
    recv_x, recv_y = np.meshgrid(coord_x, coord_y)
    groups = xr_transmmiton_data.sizes['group']
    temp_array = np.zeros(shape=(groups,len(recv_x), len(recv_y)))
    
    for group_num in range(groups):
        trans_x , trans_y, trans_power = xr_transmmiton_data.sel(group=group_num).values
        recv_power = trans_power + 20 * np.log10(lamda/(4 * pi)) - 10 * r_η * np.log10( np.sqrt(np.square(recv_x-trans_x)
               +np.square(recv_y-trans_y)) )
        temp_array[group_num,:,:] = recv_power
    
    spectrum_data = xr.DataArray(temp_array,
                             dims=('group','y', 'x'),
                            coords={'x':coord_x , 'y':coord_y})
        
    return spectrum_data

def stan_data_gen(xr_transmmition_data, xr_sensor_data):
    stan_data = {}
    stan_data['D'] = xr_transmmition_data.sizes['group']
    stan_data['N'] = xr_sensor_data.sizes['x'] * xr_sensor_data.sizes['y']
    x,y = np.meshgrid(xr_sensor_data.x.values, xr_sensor_data.y.values)
    stan_data['recv_x'] = list(x.flatten())
    stan_data['recv_y'] = list(y.flatten())
    
    stan_data['tran_power'] = list(xr_transmmition_data.sel(var='power').values)
    stan_data['tran_x'] = list(xr_transmmition_data.sel(var='x').values)
    stan_data['tran_y'] = list(xr_transmmition_data.sel(var='y').values)
    
    stan_data['tran_recv_distance'] = []
    stan_data['Y'] = []
    
    for group_num in range(xr_sensor_data.sizes['group']):
        trans_x , trans_y, _ = xr_transmmition_data.sel(group=group_num).values
        distances = np.sqrt(np.square( x - trans_x) + np.square(y-trans_y))
        
        stan_data['tran_recv_distance'].append(list(distances.flatten()))
        stan_data['Y'].append(list(xr_sensor_data.sel(group=group_num).values.flatten()))
        
    return stan_data

#  生成数据

## 数据生成函数定义

In [18]:
def data_generation(group_num, trans_power, gp_gen_model, random_seed, x_coord_nums=20, y_coord_nums=20,η=3.5, alpha_true=8, rho_true=100, sigma_true=0.2):
    '''
    生成若干组（group_num）发送数据和接收数据,接收数据是在空间均匀分布的点（由x_coord_nums和y_coord_nums确定坐标轴上取点个数）
    Parameters
        ----------
        group_num: int,生成数据的组数
        trans_power: 可迭代的功率值，可以是列表，长度=group_num
        random_seed: int,随机数种子
        gp_gen_model: 用于生成空间高斯过程数据的stan模型,可以是'gen_exp' 'gen_matern32'（在models文件夹中）
        x_coord_nums: 20
        y_coord_nums: 20       
        η: 自由空间传播损耗因子
        alpha_true: 3
        rho_true: 100
        sigma_true: 0.2
       
    Returns
        ----------
        data:dict
          'trans_data': xarray格式,dim = (group, var:[x,y,power]),每组对应一个发送点的x，y坐标和发送功率值
          ’true_recv_power‘: xarray格式，dim=('group','y', 'x'),表示对应trans_data某组坐标（x，y）处的接收功率，不考虑误差
          ’sensored_power':xarray格式，dim=('group','y', 'x')，表示在（x，y）处设置感知设备的接收功率，考虑误差       
    '''
    
    # 设置参数值
    coord_x = np.arange(0.5, 500, 499.5//x_coord_nums + 1 )
    coord_y = np.arange(-499.5, 0, 499.5//y_coord_nums + 1)

    x,y = np.meshgrid(coord_x, coord_y)
    x_total = []
    for x, y in zip(x.flatten(), y.flatten()):
        x_total.append([x,y])

    simu_data = dict(N=x_coord_nums * y_coord_nums,
                     x=x_total, alpha=alpha_true,
                     rho=rho_true,
                     sigma=sigma_true)
    
    # 读入生成空间相关数的stan模型
    model = StanModel_load(gp_gen_model)
    gp_data_gen = model.sampling(data=simu_data,
                                      iter=1,
                                      chains=group_num, 
                                      algorithm="Fixed_param")
    
    gp_data = gp_data_gen.extract()
    
    # 生成发送数据
    trans_data = gen_transmmition_data(group_num, random_seed, trans_power)
    
    # 生成自由空间传播的数据
    recv_data_free_loss = recv_power(trans_data, coord_x, coord_y, η)
    
    # 生成阴影衰落数据
    true_recv_power = recv_data_free_loss + gp_data['f'].reshape(group_num, x_coord_nums, y_coord_nums)
    
    # 生成感知数据
    sensored_power = recv_data_free_loss + gp_data['y'].reshape(group_num, x_coord_nums, y_coord_nums)
        
    data={}
    data['trans_data'] = trans_data
    data['recv_data_free_loss'] = recv_data_free_loss
    data['true_recv_power'] = true_recv_power
    data['sensored_power'] = sensored_power
    return data
    

## ——————华丽的分割线—————— Warning：以下数据以及生成，不要重复执行

### 1.生成10组数据

In [4]:
data_10groups  = data_generation(group_num=10,
                           trans_power=30*np.ones(10), 
                           gp_gen_model='gen_matern32',
                           random_seed=13)
data_10groups.keys()

Using cached StanModel:gen_matern32


dict_keys(['trans_data', 'recv_data_free_loss', 'true_recv_power', 'sensored_power'])

In [5]:
stan_data_10groups = stan_data_gen(data_10groups['trans_data'], data_10groups['sensored_power'][:,::2,::2])

In [6]:
StanData_cache(data_10groups, 'data_10groups')
StanData_cache(stan_data_10groups, 'stan_data_10groups')

DATA cached as:data_10groups.pkl
DATA cached as:stan_data_10groups.pkl


### 2.生成20组数据

In [7]:
data_20groups  = data_generation(group_num=20,
                           trans_power=30*np.ones(20), 
                           gp_gen_model='gen_matern32',
                           random_seed=1234)
data_20groups.keys()

Using cached StanModel:gen_matern32


dict_keys(['trans_data', 'recv_data_free_loss', 'true_recv_power', 'sensored_power'])

In [8]:
stan_data_20groups = stan_data_gen(data_20groups['trans_data'], data_20groups['sensored_power'][:,::2,::2])

In [9]:
StanData_cache(data_20groups, 'data_20groups')
StanData_cache(stan_data_20groups, 'stan_data_20groups')

DATA cached as:data_20groups.pkl
DATA cached as:stan_data_20groups.pkl


### 3.生成100组数据

In [7]:
data_100groups  = data_generation(group_num=100,
                           trans_power=30*np.ones(100), 
                           gp_gen_model='gen_matern32',
                           random_seed=30)
data_100groups.keys()

Using cached StanModel:gen_matern32


dict_keys(['trans_data', 'recv_data_free_loss', 'true_recv_power', 'sensored_power'])

In [8]:
stan_data_100groups = stan_data_gen(data_100groups['trans_data'], data_100groups['sensored_power'][:,::2,::2])

In [9]:
StanData_cache(data_100groups, 'data_100groups')
StanData_cache(stan_data_100groups, 'stan_data_100groups')

DATA cached as:data_100groups.pkl
DATA cached as:stan_data_100groups.pkl


### 4.生成1000组数据

In [10]:
data_1000groups  = data_generation(group_num=1000,
                           trans_power=30*np.ones(1000), 
                           gp_gen_model='gen_matern32',
                           random_seed=1234)
data_1000groups.keys()

Using cached StanModel:gen_matern32


dict_keys(['trans_data', 'recv_data_free_loss', 'true_recv_power', 'sensored_power'])

In [11]:
stan_data_1000groups = stan_data_gen(data_1000groups['trans_data'], data_1000groups['sensored_power'][:,::2,::2])

In [12]:
StanData_cache(data_1000groups, 'data_1000groups')
StanData_cache(stan_data_1000groups, 'stan_data_1000groups')

DATA cached as:data_1000groups.pkl
DATA cached as:stan_data_1000groups.pkl


### 数据量成2 的 n 次方增加：8， 16， 32， 64， 128， 256， 512， 1024

In [19]:
for n in range(3,11):
    group_num = 2**n
    data = data_generation(group_num = group_num, trans_power=30*np.ones(group_num),
                          gp_gen_model='gen_matern32',
                          random_seed = group_num)
    stan_data = stan_data_gen(data['trans_data'], data['sensored_power'][:,::2,::2])
    data_name = 'data_' + str(group_num) + 'groups'
    stan_data_name = 'stan_data_' + str(group_num) + 'groups'
    StanData_cache(data, data_name)
    StanData_cache(stan_data, stan_data_name)

Using cached StanModel:gen_matern32
DATA cached as:data_8groups.pkl
DATA cached as:stan_data_8groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_16groups.pkl
DATA cached as:stan_data_16groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_32groups.pkl
DATA cached as:stan_data_32groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_64groups.pkl
DATA cached as:stan_data_64groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_128groups.pkl
DATA cached as:stan_data_128groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_256groups.pkl
DATA cached as:stan_data_256groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_512groups.pkl
DATA cached as:stan_data_512groups.pkl
Using cached StanModel:gen_matern32
DATA cached as:data_1024groups.pkl
DATA cached as:stan_data_1024groups.pkl


In [17]:
20 * np.log10(3e8/700e6/(4 * pi))

-29.343732986333816

In [14]:
10**(30/20)

31.622776601683793